In [1]:
import conf
import json
import time
import math
import statistics
from boltiot import Sms, Bolt, Email


def compute_bounds(history_data, frame_size, factor):
    if len(history_data) < frame_size:
        return None

    if len(history_data) > frame_size:
        history_data = history_data[-frame_size:]

    mean = statistics.mean(history_data)
    variance = statistics.variance(history_data)
    deviation = factor * math.sqrt(variance)
    high_bound = history_data[-1] + deviation
    low_bound = history_data[-1] - deviation
    print("High bound = ", high_bound)
    print("Low bound = ", low_bound)
    return [high_bound, low_bound]


def send_alerts(temp, message):
    try:
        sms_response = sms.send_sms(message)
        email_response = mailer.send_email("Alert", message)
        print("This is the response for SMS", sms_response)
        print("This is the response for EMAIL", email_response)
    except Exception as e:
        print("Error sending alerts:", e)


def get_sensor_value():
    try:
        response = mybolt.analogRead('A0')
        data = json.loads(response)
        sensor_value = int(data['value'])
        return sensor_value
    except Exception as e:
        print("Error reading sensor value:", e)
        return None


mybolt = Bolt(conf.API_KEY, conf.DEVICE_ID)
sms = Sms(conf.SSID, conf.AUTH_TOKEN, conf.TO_NUMBER, conf.FROM_NUMBER)
mailer = Email(conf.MAILGUN_API_KEY, conf.SANDBOX_URL, conf.SENDER_EMAIL, conf.RECIPIENT_EMAIL)
history_data = []

while True:
    sensor_value = get_sensor_value()
    if sensor_value is None:
        time.sleep(5)
        continue

    sensor_value_f = int((sensor_value / 10.24) * 9 / 5 + 32)
    print(f"The current temp is {sensor_value_f} degrees F, and the sensor value is {sensor_value}")

    bound = compute_bounds(history_data, conf.FRAME_SIZE, conf.MUL_FACTOR)
    if not bound:
        required_data_count = conf.FRAME_SIZE - len(history_data)
        print(f"I will collect {required_data_count} more data point(s) before I begin.")
        history_data.append(sensor_value)
        time.sleep(5)
        continue

    if sensor_value > bound[0]:
        message = f"Temp has increased. The current temperature is {sensor_value_f} degrees F."
        print("The temp has INCREASED. Sending SMS & email.")
        send_alerts(sensor_value_f, message)

    elif sensor_value < bound[1]:
        message = f"Temp has decreased. The current temperature is {sensor_value_f} degrees F."
        print("The temp has DECREASED. Sending SMS & email.")
        send_alerts(sensor_value_f, message)

    history_data.append(sensor_value)
    time.sleep(5)


The current temp is 75 degrees F, and the sensor value is 245
I will collect  4  more data point(s) before I begin.
The current temp is 66 degrees F, and the sensor value is 196
I will collect  3  more data point(s) before I begin.
The current temp is 73 degrees F, and the sensor value is 235
I will collect  2  more data point(s) before I begin.
The current temp is 119 degrees F, and the sensor value is 497
I will collect  1  more data point(s) before I begin.
The current temp is 73 degrees F, and the sensor value is 235
High bound =  854.1535909101293
Low bound =  139.8464090898707
The current temp is 68 degrees F, and the sensor value is 207
High bound =  575.3479836872843
Low bound =  -105.34798368728434
The current temp is 33 degrees F, and the sensor value is 9
High bound =  249.0
Low bound =  165.0
The temp  has decreased. Sending SMS & email.
This is the response for SMS  <Twilio.Api.V2010.MessageInstance account_sid=AC4118536bc4a7d6aa77e743a280fb1b4f sid=SM7f7704dea470d82ad24d6

KeyboardInterrupt: 